In [2]:
#Imports
import pandas as pd
from IPython.display import Markdown, display

Pretty printing

In [3]:
def printBold(string):
    display(Markdown('**' + string + '**'))
    
#def printColor():
#     display(Markdown('<span style="color:blue">blue</span>'))

In [51]:
train = pd.read_json('../data/squad-v1/train-v1.1.json', orient='column')
dev = pd.read_json('../data/squad-v1/dev-v1.1.json', orient='column')

In [57]:
df = pd.concat([train, dev], ignore_index=True)

In [58]:
df.head()

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1
3,"{'title': 'Genocide', 'paragraphs': [{'context...",1.1
4,"{'title': 'Antibiotics', 'paragraphs': [{'cont...",1.1


## Dataset size

In [62]:
titlesCount = len(df['data'])
totalParagraphsCount = 0
totalQuestionsCount = 0

for titleId in range(titlesCount):
    paragraphsCount = len(df['data'][titleId]['paragraphs'])
    totalParagraphsCount += paragraphsCount
    
    for paragraphId in range(paragraphsCount):
        questionsCount = len(df['data'][titleId]['paragraphs'][paragraphId]['qas'])
        
        totalQuestionsCount += questionsCount
        
print('Titles', titlesCount)
print('Paragraphs', totalParagraphsCount)
print('Questions', totalQuestionsCount)

Titles 490
Paragraphs 20963
Questions 98169


## Titles

In [63]:
titles = []
for titleId in range(len(df['data'])):
    titles.append(df['data'][titleId]['title'])
    
titles

['University_of_Notre_Dame',
 'Beyoncé',
 'Montana',
 'Genocide',
 'Antibiotics',
 'Frédéric_Chopin',
 'Sino-Tibetan_relations_during_the_Ming_dynasty',
 'IPod',
 'The_Legend_of_Zelda:_Twilight_Princess',
 'Spectre_(2015_film)',
 '2008_Sichuan_earthquake',
 'New_York_City',
 'To_Kill_a_Mockingbird',
 'Solar_energy',
 'Tajikistan',
 'Anthropology',
 'Portugal',
 'Kanye_West',
 'Buddhism',
 'American_Idol',
 'Dog',
 '2008_Summer_Olympics_torch_relay',
 'Alfred_North_Whitehead',
 'Financial_crisis_of_2007%E2%80%9308',
 'Saint_Barth%C3%A9lemy',
 'Genome',
 'Comprehensive_school',
 'Republic_of_the_Congo',
 'Prime_minister',
 'Institute_of_technology',
 'Wayback_Machine',
 'Dutch_Republic',
 'Symbiosis',
 'Canadian_Armed_Forces',
 'Cardinal_(Catholicism)',
 'Iranian_languages',
 'Lighting',
 'Separation_of_powers_under_the_United_States_Constitution',
 'Architecture',
 'Human_Development_Index',
 'Southern_Europe',
 'BBC_Television',
 'Arnold_Schwarzenegger',
 'Plymouth',
 'Heresy',
 'Warsa

Titles are pretty random. Seems to be a lot of locations like countries and cities but not nearly enough to afford splitting the dataset.

## Questions

In [64]:
def showQuestion(titleId, paragraphId, questionId):

    title = df['data'][titleId]['title']
    paragraph = df['data'][titleId]['paragraphs'][paragraphId]['context']
    question = df['data'][titleId]['paragraphs'][paragraphId]['qas'][questionId]['question']
    answer = df['data'][titleId]['paragraphs'][paragraphId]['qas'][questionId]['answers'][0]['text']

    printBold('Title')
    print(title)
    printBold('Paragraph')
    print(paragraph)
    printBold('Question')
    print(question)
    printBold('Answer')
    print(answer)

In [65]:
titleId = 16
paragraphId = 4 
questionId = 1

showQuestion(titleId, paragraphId, questionId)

**Title**

Portugal


**Paragraph**

With the Conference of Berlin of 1884, Portuguese Africa territories had their borders formally established on request of Portugal in order to protect the centuries-long Portuguese interests in the continent from rivalries enticed by the Scramble for Africa. Portuguese Africa's cities and towns like Nova Lisboa, Sá da Bandeira, Silva Porto, Malanje, Tete, Vila Junqueiro, Vila Pery and Vila Cabral were founded or redeveloped inland during this period and beyond. New coastal towns like Beira, Moçâmedes, Lobito, João Belo, Nacala and Porto Amélia were also founded. Even before the turn of the 20th century, railway tracks as the Benguela railway in Angola, and the Beira railway in Mozambique, started to be built to link coastal areas and selected inland regions.


**Question**

When was the Conference of Berlin held?


**Answer**

1884


My main assumption is that the sentence the answer is contained within could be transformed into a question. Let's see how true is that in light of the 87599 questions we have.

In [66]:
#def containedInText()
    #for each word in the question check if it's contained in the sentence
    
#def extractSentence()

#or the parapragh

#return fraction for containment in the range [0;1]